# Amazon Personalize

[Amazon Personalize](https://docs.aws.amazon.com/personalize/latest/dg/what-is-personalize.html) 是一项完全托管的机器学习服务，利用您的数据为用户生成物品推荐。它还可以根据用户对特定物品或物品元数据的偏好生成用户分段。

本笔记本介绍如何使用 Amazon Personalize Chain。在开始使用下面的笔记本之前，您需要一个 Amazon Personalize campaign_arn 或 recommender_arn。

以下是一个[教程](https://github.com/aws-samples/retail-demo-store/blob/master/workshop/1-Personalization/Lab-1-Introduction-and-data-preparation.ipynb)，指导如何在 Amazon Personalize 上设置一个 campaign_arn/recommender_arn。一旦设置了 campaign_arn/recommender_arn，您就可以在 langchain 生态系统中使用它。

## 1. 安装依赖库

In [ ]:
# 安装boto3库
!pip install boto3

## 2. 样本使用案例

### 2.1 [使用案例-1] 设置 Amazon Personalize 客户端并获取推荐结果

In [ ]:
from langchain_experimental.recommenders import AmazonPersonalize  # 导入AmazonPersonalize推荐系统模块

recommender_arn = "<insert_arn>"  # 设置推荐系统的ARN

client = AmazonPersonalize(  # 创建AmazonPersonalize客户端实例
    credentials_profile_name="default",  # 使用默认凭证配置文件
    region_name="us-west-2",  # 设置区域为us-west-2
    recommender_arn=recommender_arn,  # 设置推荐系统的ARN
)
client.get_recommendations(user_id="1")  # 获取用户ID为"1"的推荐结果

### 2.2 [使用案例-2] 调用个性化链以总结结果

In [ ]:
from langchain.llms.bedrock import Bedrock  # 导入Bedrock类
from langchain_experimental.recommenders import AmazonPersonalizeChain  # 导入AmazonPersonalizeChain类

bedrock_llm = Bedrock(model_id="anthropic.claude-v2", region_name="us-west-2")  # 创建Bedrock实例，指定模型ID和地区名称

# 创建个性化推荐链
# 如果不需要摘要，可以使用return_direct=True
chain = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=False  # 从Bedrock实例创建AmazonPersonalizeChain实例
)
response = chain({"user_id": "1"})  # 调用推荐链，传入用户ID
print(response)  # 打印推荐结果

### 2.3 [使用案例-3] 使用您自己的提示调用 Amazon Personalize 链

In [ ]:
from langchain.prompts.prompt import PromptTemplate

RANDOM_PROMPT_QUERY = """
You are a skilled publicist. Write a high-converting marketing email advertising several movies available in a video-on-demand streaming platform next week, 
    given the movie and user information below. Your email will leverage the power of storytelling and persuasive language. 
    The movies to recommend and their information is contained in the <movie> tag. 
    All movies in the <movie> tag must be recommended. Give a summary of the movies and why the human should watch them. 
    Put the email between <email> tags.

    <movie>
    {result} 
    </movie>

    Assistant:
    """

# 创建一个 PromptTemplate 实例，传入输入变量和模板
RANDOM_PROMPT = PromptTemplate(input_variables=["result"], template=RANDOM_PROMPT_QUERY)

# 使用 AmazonPersonalizeChain 类从 bedrock_llm 创建一个链式模型实例
chain = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=False, prompt_template=RANDOM_PROMPT
)

# 运行链式模型，传入用户ID和物品ID
chain.run({"user_id": "1", "item_id": "234"})

### 2.4 [使用案例-4] 在顺序链中调用Amazon Personalize

In [ ]:
from langchain.chains import LLMChain, SequentialChain

RANDOM_PROMPT_QUERY_2 = """
你是一位熟练的公关专员。写一封高转化率的营销邮件，广告一家视频点播流媒体平台下周提供的多部电影，给出以下电影和用户信息。你的邮件将利用叙事和有说服力的语言的力量。你希望这封邮件能给用户留下深刻印象，让他们感到吸引。
推荐的电影及其信息包含在 <movie> 标签中。
必须推荐 <movie> 标签中的所有电影。给出电影的摘要以及为什么人类应该观看它们。
将邮件放在 <email> 标签之间。

<movie>
{result}
</movie>

助手：
"""

RANDOM_PROMPT_2 = PromptTemplate(
    input_variables=["result"], template=RANDOM_PROMPT_QUERY_2
)
personalize_chain_instance = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=True
)
random_chain_instance = LLMChain(llm=bedrock_llm, prompt=RANDOM_PROMPT_2)
overall_chain = SequentialChain(
    chains=[personalize_chain_instance, random_chain_instance],
    input_variables=["user_id"],
    verbose=True,
)
overall_chain.run({"user_id": "1", "item_id": "234"})

### 2.5 [使用案例-5] 调用 Amazon Personalize 并检索元数据

In [ ]:
# 设置推荐器的 ARN
recommender_arn = "<insert_arn>"

# 设置元数据列的名称
metadata_column_names = [
    "<insert metadataColumnName-1>",
    "<insert metadataColumnName-2>",
]

# 将元数据列名称放入字典中
metadataMap = {"ITEMS": metadata_column_names}

# 创建 Amazon Personalize 客户端
client = AmazonPersonalize(
    credentials_profile_name="default",
    region_name="us-west-2",
    recommender_arn=recommender_arn,
)

# 获取推荐结果
client.get_recommendations(user_id="1", metadataColumns=metadataMap)

### 2.6 [使用案例 6] 使用返回的元数据调用 Personalize Chain 来总结结果



In [ ]:
bedrock_llm = Bedrock(model_id="anthropic.claude-v2", region_name="us-west-2")

# 创建个性化链
# 如果不需要摘要，可以使用 return_direct=True
chain = AmazonPersonalizeChain.from_llm(
    llm=bedrock_llm, client=client, return_direct=False
)
response = chain({"user_id": "1", "metadata_columns": metadataMap})
print(response)